# 1) Imports và cấu hình ban đầu

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time, pandas as pd


**selenium** / **webdriver**: thư viện điều khiển trình duyệt.

**By**: enum để chỉ loại selector (By.CSS_SELECTOR, By.XPATH, ...).

**Service + ChromeDriverManager: ChromeDriverManager().install()** sẽ tự tải binary chromedriver tương thích.

**Options**: để set các flag cho Chrome.

**time** dùng sleep, **pandas** để lưu dữ liệu vào DataFrame / Excel.

# 2) Cấu hình Chrome (opts)

In [ ]:
opts = Options()
# opts.add_argument("--headless")   # uncomment nếu muốn chạy ẩn
opts.add_argument("--no-sandbox")
opts.add_argument("--disable-dev-shm-usage")
opts.add_argument("--disable-blink-features=AutomationControlled")
opts.add_experimental_option("excludeSwitches", ["enable-automation"])
opts.add_experimental_option('useAutomationExtension', False)
opts.add_argument("user-agent=Mozilla/5.0 ... Chrome/120.0.0.0 Safari/537.36")


**headless**: chạy không mở cửa sổ UI (bị comment).

**--no-sandbox, --disable-dev-shm-usage**: thường dùng khi chạy trong container/Docker để tránh lỗi.

**--disable-blink-features=AutomationControlled, excludeSwitches, useAutomationExtension**: cố gắng che bớt dấu hiệu trình duyệt bị điều khiển (chú ý: không đảm bảo 100%).

**user-agent**: thay đổi user agent của trình duyệt để trông giống trình duyệt thực.

# 3) Khởi tạo webdriver

In [ ]:
drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

**ChromeDriverManager().install()** tự tải chromedriver phù hợp.

**drv** là instance(đối tượng) WebDriver dùng cho tất cả thao tác (get, find_element...).

# 4) Các URL category

In [ ]:
urls = {
    "Điện thoại": "https://www.lazada.vn/catalog/?q=điện%20thoại",
    "Quần áo": "https://www.lazada.vn/catalog/?q=quần%20áo",
    "Tủ lạnh": "https://www.lazada.vn/catalog/?q=tủ%20lạnh",
    "Tivi": "https://www.lazada.vn/catalog/?q=ti%20vi"
}


**urls** là dict mapping tên category -> URL tìm kiếm (đã mã hóa dấu cách bằng %20).

# 5) Hằng nhận diện icon sao

In [ ]:
FULL, HALF = "Dy1nx", "JhbDv"  # keyword nhận diện icon sao

2 chuỗi này là một phần trong class của thẻ icon sao (class bị obfuscated trên Lazada).

**parse_rating** sẽ kiểm tra xem tên class có chứa chuỗi này để đếm sao đầy hoặc nửa sao.

Chú ý: class obfuscated hay thay đổi => có thể không bền.

# 6) Hàm cuộn trang

In [ ]:
def scroll(times=4, pause=2):
    last = drv.execute_script("return document.body.scrollHeight")
    for _ in range(times):
        drv.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        new = drv.execute_script("return document.body.scrollHeight")
        if new == last: break
        last = new


Lấy **document.body.scrollHeight** để biết chiều cao trang.

Lặp **times** lần: cuộn xuống cuối trang, chờ **pause** giây để nội dung động load.

Nếu chiều cao không tăng (không có nội dung mới) thì **break** sớm.

Mục đích: kích hoạt lazy-load / infinite scroll để sản phẩm hiện ra.

# 7) Hàm parse_rating(p)

In [ ]:
def parse_rating(p):
    stars = p.find_elements(By.CSS_SELECTOR, "i._9-ogB")
    if not stars: return "N/A"
    full = sum(FULL in s.get_attribute("class") for s in stars)
    half = sum(HALF in s.get_attribute("class") for s in stars)
    return full + 0.5 * half if (full or half) else "N/A"

**p** là WebElement đại diện 1 sản phẩm.

**find_elements(By.CSS_SELECTOR, "i._9-ogB")**: tìm tất cả icon sao (selector dựa trên class **_9-ogB**).

Nếu không tìm thấy icon => trả **"N/A"**.

Đếm số full stars: **FULL in class_attribute**.

Đếm số half stars: **HALF in class_attribute**.

Trả tổng điểm: **full + 0.5 * half**, hoặc **"N/A"** nếu cả hai đều 0.

Giới hạn: nếu structure thay đổi hoặc icon không phải kiểu này thì kết quả sai.

# 8) Hàm crawl_page(url, category, page_num)

In [ ]:
def crawl_page(url, category, page_num):
    page_url = url + f"&page={page_num}"
    drv.get(page_url)
    time.sleep(3)
    scroll()

    products = drv.find_elements(By.CSS_SELECTOR, "div.Bm3ON")
    print(f"[{category}] Page {page_num} -> {len(products)} sản phẩm")
    rows = []
    for p in products:
        try:
            name = p.find_element(By.CSS_SELECTOR, "div.RfADt a").get_attribute("title")
        except: name = "N/A"

        try: price = p.find_element(By.CSS_SELECTOR, "span.ooOxS").text
        except: price = "N/A"

        try: sold = p.find_element(By.XPATH, ".//span[contains(text(),'Đã bán')]").text
        except: sold = "N/A"

        rating = parse_rating(p)

        rows.append({
            "Category": category,
            "Page": page_num,
            "Name": name,
            "Price": price,
            "SoldCount": sold,
            "Rating": rating
        })
    return rows


**page_url** = url + f"&page={page_num}": nối thêm tham số page. (Ok vì url đã có ?q=....)

**drv.get(page_url)**: mở trang.

**time.sleep(3)**: đợi trang tải (có thể thay bằng WebDriverWait để chính xác hơn).

**scroll()** để load thêm sản phẩm.

**drv.find_elements(By.CSS_SELECTOR, "div.Bm3ON")**: lấy danh sách container mỗi sản phẩm (selector dựa trên class).

Trong vòng lặp **for p in products**:

**(**

   Lấy **name**: tìm **div.RfADt** a rồi get_attribute("title") (tên thường nằm ở attribute title).

   Lấy **price**: **span.ooOxS.text**

   Lấy **sold**: dùng **XPath** tìm span chứa chữ 'Đã bán'.

   **rating = parse_rating(p)** -> gọi hàm phân tích sao.

   Mỗi sản phẩm được lưu dưới dạng dict và append vào **rows**.
   
**)**

Trả về **rows** (danh sách dict) cho page đó.

# 9) Vòng lặp chính crawl 3 trang mỗi category

In [ ]:
all_rows = []
for cat, url in urls.items():
    for page in range(1, 4):   # page 1 -> 3
        all_rows.extend(crawl_page(url, cat, page))


Vòng lặp này sẽ chạy page 1..3 cho mỗi category.

# 10) Lưu file Excel

In [ ]:
if all_rows:
    df = pd.DataFrame(all_rows)
    df.to_excel("lazada_products_pages.xlsx", index=False)
    print("✅ Lưu xong lazada_products_pages.xlsx")
else:
    print("❌ Không lấy được dữ liệu")

Nếu **all_rows** có dữ liệu thì tạo **DataFrame** và **to_excel**.

**.to_excel** cần package **openpyxl** (pandas tự chọn engine nếu **.xlsx**).

Tên file: **"lazada_products_pages.xlsx".**